In [17]:


def find_matching_result(df, suggestion):
    if suggestion:
        # Create the suggestion string in the same format as the formulation strings
        suggestion_str = f'The formulation is Powderkg = {suggestion["powderkg"]}, wc = {suggestion["wc"]}, materials = {suggestion["materials"]}, curing = {suggestion["curing"]}'

        # Look for a match in the DataFrame
        match = df[df["Formulation"].str.lower() == suggestion_str.lower()]

        # If a match was found, return the lab result
        if not match.empty:
            return match.iloc[0]["Strength"]

    # If no match was found, return None
    return None

def parse_solution(response):
    # Initialize a dictionary to hold the solution
    solution = {}

    # Find matches for each key
    keys = ['powderkg', 'wc', 'materials', 'curing']
    for key in keys:
        # Use regex to find the key followed by = and the value
        match = re.search(fr'{key} = (.*?)(,|$)', response, re.IGNORECASE)
        if match:
            # If a match was found, add it to the solution dictionary
            value = match.group(1).strip()
            
            # Remove trailing period from the 'curing' value
            if key == 'curing':
                value = value.rstrip('.')
                
            solution[key] = value
        else:
            return None  # If any key wasn't found, return None

    # Return the solution dictionary if all keys were found
    return solution

    
def format_response_to_model(lab_result):
    """
    Given a lab result, format a response message to the model.
    """
    return f"We've achieved a compressive strength of {lab_result['fc_28d_Lab_validation']} MPa. Let's try to do better!"

def parse_materials(materials_str):
    match = re.search(r'(\d+)/(\d+) FA/GGBFS', materials_str)
    if match:
        return int(match.group(1)) / (int(match.group(1)) + int(match.group(2)))
    else:
        return None
    
def parse_curing(materials_str):
    if "Ambient curing" in materials_str:
        return "ambient"
    elif "Heat curing" in materials_str:
        return "oven"
    else:
        return None

def load_data(csv_path):
    df = pd.read_csv(csv_path)
    df['FA_GGBFS_ratio'] = df['Materials'].apply(parse_materials)
    df['curing'] = df['Materials'].apply(parse_curing)  # Add this line
    return df

df = load_data('Data/DiscoveryData_Sample.csv')

# Initialize empty DataFrame
formulation_df = pd.DataFrame(columns=["Formulation", "Strength"])

# Loop through each row in the original data
for idx, row in df.iterrows():
    
    # Get necessary attributes from row
    powder = row["Powderkg"]
    wc = row["WC"]
    materials = row["Materials"]

    # Extract Fly Ash/GGBFS ratio
    fa_ggbfs = materials.split(",")[0].split("-")[1]
    
    # Extract curing method
    curing_method = materials.split(",")[-1].strip()

    # Remove unwanted string from curing method
    curing_method = curing_method.replace(" (Rao et al. 2018)", "")
    curing_method = curing_method.replace(" (Rao et al.)", "")
    
    # Compressive strength
    strength = row["fc_28dGroundTruth"]
    
    # Create formulation string in the same format as the model's output
    formulation = f'The formulation is Powderkg = {powder}, wc = {wc}, materials = {fa_ggbfs}, curing = {curing_method}'
    
    # Append the formulation and its respective strength to the new DataFrame
    new_row = pd.DataFrame({"Formulation": [formulation], "Strength": [strength]})
    formulation_df = pd.concat([formulation_df, new_row], ignore_index=True)


def handle_openai_error(exception):
    if isinstance(exception, openai.error.RateLimitError):
        print(f"Rate limit error. Will retry after {exception.wait_seconds} seconds.")
        time.sleep(exception.wait_seconds)
    elif isinstance(exception, openai.error.InvalidRequestError):
        print(f"Invalid request: {str(exception)}")
    elif isinstance(exception, openai.error.AuthenticationError):
        print(f"Authentication error: {str(exception)}")
    elif isinstance(exception, openai.error.ServiceUnavailableError):
        print(f"Service unavailable error. Retrying after a delay...")
        time.sleep(5)  # Sleep for 5 seconds before retrying
    elif isinstance(exception, openai.error.APIError):
        print(f"API error: {str(exception)}. Retrying after a delay...")
        time.sleep(5)  # Sleep for 5 seconds before retrying
    elif isinstance(exception, openai.error.Timeout):
        print(f"Timeout error: {str(exception)}. Retrying after a longer delay...")
        time.sleep(10)  # Sleep for 10 seconds before retrying
    else:
        raise exception
# -> here we also set the API parameters, such as temperature, etc.

def call_openai_api(messages,temp, max_retries=5, delay=5):
    for i in range(max_retries):
        try:
            response = openai.ChatCompletion.create(
                model= model_dropdown.value,
                temperature=temp,
                messages=messages,
                max_tokens=250,
                n=1
            )
            return response
        except openai.error.OpenAIError as e:
            handle_openai_error(e)
            if i < max_retries - 1:  # i is zero indexed
                time.sleep(delay)  # wait before trying again
                continue
            else:
                raise
                
# Load the text from the file
if  prompt_dropdown.value == 'None':
    with open('prompts_ID_none.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Generic':
    with open('prompts_ID_generic.txt', 'r') as f:
        lines = f.read().splitlines()
elif prompt_dropdown.value == 'Specific':
    with open('prompts_ID_specific.txt', 'r') as f:
        lines = f.read().splitlines()
    
# Store the contents in separate variables
#instructions_text = lines[0]
system_role_text = lines[0]
context_text = lines[1]
iterate_text = lines[2]


# Create the widgets with the loaded text
layout = widgets.Layout(width='auto', height='200px')  # adjust the height and width as needed
#instructions_prompt = widgets.Textarea(value=instructions_text, description='Instructions:', layout=layout)
system_role_prompt = widgets.Textarea(value=system_role_text, description='System Role:', layout=layout)
context_prompt = widgets.Textarea(value=context_text, description='Design Rules:', layout=layout)
iterate_prompt = widgets.Textarea(value=iterate_text, description='Feedback:', layout=layout)


def update_file(button):
    if  prompt_dropdown.value == 'None':
        with open('prompts_ID_none.txt', 'w') as f:
            #f.write(instructions_prompt.value + '\n')
            f.write(system_role_prompt.value + '\n')
            f.write(context_prompt.value + '\n')
            f.write(iterate_prompt.value + '\n')

    elif prompt_dropdown.value == 'Generic':
        with open('prompts_ID_generic.txt', 'w') as f:
            #f.write(instructions_prompt.value + '\n')
            f.write(system_role_prompt.value + '\n')
            f.write(context_prompt.value + '\n')
            f.write(iterate_prompt.value + '\n')

    elif prompt_dropdown.value == 'Specific':
        with open('prompts_ID_specific.txt', 'w') as f:
            #f.write(instructions_prompt.value + '\n')
            f.write(system_role_prompt.value + '\n')
            f.write(context_prompt.value + '\n')
            f.write(iterate_prompt.value + '\n')

        
    print("💽Saved!") 

# Link the button to the update_file function
save_button = widgets.Button(description="Save Prompts",
                             layout=widgets.Layout(width='100%', height='30px'))

# Link the button to the update_file function
save_button.on_click(update_file)

# Change the color of the button to green
save_button.style.button_color = 'lightgreen'

# Run to display the text box widgets and the save button

# Set the 'flex' property for each widget inside main_layout
#instructions_prompt.layout.flex = '2'  # Adjust the value as needed
iterate_prompt.layout.flex = '2'       # Adjust the value as needed

#Instructions_box = widgets.HBox([instructions_prompt],layout=widgets.Layout(width='100%', height='100px'))
Iterate_box = widgets.HBox([iterate_prompt],layout=widgets.Layout(width='100%', height='100px'))

title = widgets.HTML("<h2>Edit Prompts</h2>")
Prompts = widgets.VBox([title, Iterate_box, context_prompt,system_role_prompt, save_button])

temperatures_str = model_temperatures.value
# Split the string using commas as a delimiter and convert to float numbers
temperatures = [float(t) for t in temperatures_str.split(',')]
budget = num_development.value
NrOfExper = num_experiments.value

desired_strength = formulation_df["Strength"].quantile(targ_quant.value/100)

num_entries_above_desired = (formulation_df["Strength"] >= desired_strength).sum()

# Print the result

print('SUMMARY\n' +'The design target is to achieve a strength of ',desired_strength, 'MPa within ',num_development.value,' development cycles.\n' +
      'The Experiment is repeated ',num_experiments.value,' times using the ',model_dropdown.value, ' model and the prompt strategy: ',prompt_dropdown.value,'.')
print("There are ", num_entries_above_desired,' formulations above or equal to desired_strength.')



NameError: name 'pd' is not defined